<a href="https://colab.research.google.com/github/EdvanilsonBS/Desafio/blob/main/Desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu

In [99]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import seaborn as sns
import pandas as pd

In [100]:
tf.__version__
'2.0.0-betal'

'2.0.0-betal'

In [101]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [102]:
logdir='log'

In [103]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0

Classes=[0,1,2,3,4,5,6,7,8,9]

In [104]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [105]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=train_images,
          y=train_labels,
          epochs=5,
          validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 97s 47ms/step - accuracy: 0.8844 - loss: 0.3759 - val_accuracy: 0.9833 - val_loss: 0.0530
Epoch 2/5
 818/1875 ━━━━━━━━━━━━━━━━━━━━ 35s 34ms/step - accuracy: 0.9825 - loss: 0.0550

In [ ]:
import numpy as np

y_true = test_labels
y_pred = np.argmax(model.predict(test_images), axis=1)


In [ ]:
classes=[0,1,2,3,4,5,6,7,8,9]

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                          index = classes,
                          columns = classes)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('true label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
modell = models.Sequential()
modell.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
modell.add(layers.MaxPool2D((2, 2)))
modell.add(layers.Conv2D(64, (3, 3), activation='relu'))
modell.add(layers.MaxPool2D((2, 2)))
modell.add(layers.Conv2D(64, (3 ,3), activation='relu'))

modell.add(layers.Flatten())
modell.add(layers.Dense(64, activation='relu'))
modell.add(layers.Dense(10, activation='softmax'))

modell.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuraracy']
)


In [ ]:
file_writer = tf.summary.create_file_writer(logdir== '/cm')

def log_confusion_matrix(epoch, logs):
test_pred = model.predict(test_images)  # Chamada correta do modelo

con_mat = tf.math.confusion_matrix(labels=test_labels, predictions=test_pred).numpy()
con_mat_norm = np.aroud(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                          index = classes
                          columns= classes)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot= True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')

buf = io.BytesIO()
plt.savefig(buf, format='png')

plt.close(figure)
buf.seek(0)
image = tf.image.decode_png(buf.getvalue(), channels=4)

image = tf.expand_dims(image, 0)

with file_writer.as_defalt():
  tf.summary.image("Confusion Matrix", image, step=epoch)

logdir='logs/images'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
from re import VERBOSE
modell.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0,
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

In [ ]:
%tensorboard --logdir logs/images
